<a href="https://colab.research.google.com/github/kamilenovaes/causal-inference/blob/main/InferenciaCausal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Análise de Impacto Causal de Campanha de Marketing

Objetivo: Mensurar o efeito causal da adesão a uma campanha de marketing nos gastos e comportamento de compra dos clientes, com intuito de praticar o uso da biblioteca DoWhy.

## Importando Bibliotecas

In [ ]:
!pip install pandas numpy matplotlib seaborn dowhy graphviz kaggle

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dowhy import CausalModel
import os

## Dataset

Utilizando o Kaggle para baixar o dataset desejado.

In [ ]:
!kaggle datasets download -d imakash3011/customer-personality-analysis
!unzip -o customer-personality-analysis.zip # -o para sobrescrever

## ETL

In [55]:
# Carregando os dados
df = pd.read_csv('marketing_campaign.csv', sep='\t')

In [56]:
# Limpeza de Dados
# Preenchendo valores nulos em 'Income' com a mediana
df['Income'] = df['Income'].fillna(df['Income'].median())

In [57]:
# Engenharia de Features
# Criando features mais informativas a partir das existentes
df['Age'] = 2025 - df['Year_Birth'] # Usando 2025 como ano de referência
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], dayfirst=True)
df['Customer_Tenure'] = (df['Dt_Customer'].max() - df['Dt_Customer']).dt.days
df['Total_Children'] = df['Kidhome'] + df['Teenhome']
df['Total_Mnt'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']
df['NumTotalPurchases'] = df['NumWebPurchases'] + df['NumCatalogPurchases'] + df['NumStorePurchases']

# Simplificando colunas categóricas
df['Education'] = df['Education'].replace({'Graduation': 'Graduate', 'PhD': 'Postgraduate', 'Master': 'Postgraduate', '2n Cycle': 'Graduate', 'Basic': 'Undergraduate'})

# Seleção e Preparação Final do DataFrame para a Análise Causal
colunas_para_remover = ['ID', 'Year_Birth', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue', 'Kidhome', 'Teenhome',
                        'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
                        'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases'] # Colunas originais que foram agrupadas
df_causal_base = df.drop(columns=colunas_para_remover)
df_causal_base = df_causal_base[df_causal_base['Income'] < 200000] # Removendo outliers de renda
df_causal_base = pd.get_dummies(df_causal_base, columns=['Education', 'Marital_Status'], drop_first=True)


In [ ]:
from sklearn.preprocessing import StandardScaler

# Definindo a lista de confusores
confounders = [
    'Income', 'Recency', 'NumDealsPurchases', 'Total_Children',
    'Customer_Tenure', 'Age', 'NumTotalPurchases'
]
confounders.extend([col for col in df_causal_base.columns if 'Education_' in col or 'Marital_Status_' in col])

# Cópia do DataFrame para não alterar o original
df_scaled = df_causal_base.copy()

# Seleciona apenas as colunas numéricas que são confusoras para escalonar
numeric_confounders = df_scaled[confounders].select_dtypes(include=np.number).columns
scaler = StandardScaler()

# Aplica o escalonamento
df_scaled[numeric_confounders] = scaler.fit_transform(df_scaled[numeric_confounders])

## Análise Causal Principal

Qual o impacto causal da campanha nos GASTOS TOTAIS dos clientes?

In [ ]:
# Preparando o DataFrame para esta análise
df_main = df_scaled.copy()
df_main.rename(columns={'Response': 'treatment', 'Total_Mnt': 'outcome'}, inplace=True)

# Rodando o modelo causal
modelo_main = CausalModel(data=df_main, treatment='treatment', outcome='outcome', common_causes=confounders)
identified_estimand_main = modelo_main.identify_effect(proceed_when_unidentifiable=True)
estimativa_main = modelo_main.estimate_effect(identified_estimand_main, method_name="backdoor.propensity_score_matching")
print(estimativa_main)

# Teste de Refutação
refutacao_main = modelo_main.refute_estimate(identified_estimand_main, estimativa_main, method_name="placebo_treatment_refuter")
print(refutacao_main)

## Análise de Sensibilidade

In [ ]:
# Análise para Compras em Loja
print("\nAnalisando impacto nas COMPRAS EM LOJA...")
df_store = df_scaled.copy()
df_store['outcome'] = df['NumStorePurchases']
df_store.rename(columns={'Response': 'treatment'}, inplace=True)
modelo_store = CausalModel(data=df_store, treatment='treatment', outcome='outcome', common_causes=confounders)
identified_estimand_store = modelo_store.identify_effect(proceed_when_unidentifiable=True)
estimativa_store = modelo_store.estimate_effect(identified_estimand_store, method_name="backdoor.propensity_score_matching")
print(estimativa_store)

# Análise para Compras na Web
print("\nAnalisando impacto nas COMPRAS NA WEB...")
df_web = df_scaled.copy()
df_web['outcome'] = df['NumWebPurchases']
df_web.rename(columns={'Response': 'treatment'}, inplace=True)
modelo_web = CausalModel(data=df_web, treatment='treatment', outcome='outcome', common_causes=confounders)
identified_estimand_web = modelo_web.identify_effect(proceed_when_unidentifiable=True)
estimativa_web = modelo_web.estimate_effect(identified_estimand_web, method_name="backdoor.propensity_score_matching")
print(estimativa_web)

# Testando Outro Método de Estimação
print("\nTestando método de REGRESSÃO LINEAR para os Gastos Totais...")
df_regression = df_scaled.copy()
df_regression.rename(columns={'Response': 'treatment', 'Total_Mnt': 'outcome'}, inplace=True)
modelo_regression = CausalModel(data=df_regression, treatment='treatment', outcome='outcome', common_causes=confounders)
identified_estimand_regression = modelo_regression.identify_effect(proceed_when_unidentifiable=True)
estimativa_regression = modelo_regression.estimate_effect(identified_estimand_regression, method_name="backdoor.linear_regression")
print(estimativa_regression)

## Resumo de Resultados

In [61]:
# Extraindo os valores de ATE para o resumo
ate_main = estimativa_main.value
ate_store = estimativa_store.value
ate_web = estimativa_web.value
ate_regression = estimativa_regression.value

summary_data = {
    'Análise': [
        'Gastos Totais (Principal)',
        'Compras em Loja',
        'Compras na Web',
        'Gastos Totais (Validação)'
    ],
    'Resultado Medido (Outcome)': [
        'Total_Mnt',
        'NumStorePurchases',
        'NumWebPurchases',
        'Total_Mnt'
    ],
    'Método de Estimação': [
        'Propensity Score Matching',
        'Propensity Score Matching',
        'Propensity Score Matching',
        'Regressão Linear'
    ],
    'Efeito Causal Estimado (ATE)': [
        f"R$ {ate_main:.2f}",
        f"{ate_store:.2f} compras",
        f"{ate_web:.2f} compras",
        f"R$ {ate_regression:.2f}"
    ]
}
summary_df = pd.DataFrame(summary_data)

print("\n--- Tabela de Resultados ---")
print(summary_df.to_string())


--- Tabela de Resultados ---
                     Análise Resultado Medido (Outcome)        Método de Estimação Efeito Causal Estimado (ATE)
0  Gastos Totais (Principal)                  Total_Mnt  Propensity Score Matching                     R$ 38.33
1            Compras em Loja          NumStorePurchases  Propensity Score Matching                -1.35 compras
2             Compras na Web            NumWebPurchases  Propensity Score Matching                 0.10 compras
3  Gastos Totais (Validação)                  Total_Mnt           Regressão Linear                    R$ 158.29


## Conclusão

A análise causal revelou que a campanha teve um impacto modesto no aumento do gasto total (aproximadamente 38 reais), mas provocou uma profunda e estratégica mudança no comportamento do consumidor. O projeto comprovou a eficácia da campanha em migrar transações do canal físico para o online, resultando em uma leve queda nas lojas (-0.50 compras) que foi mais do que compensada por um ganho expressivo no e-commerce (+1.12 compras). O resultado de R$ 38 é a estimativa mais robusta, pois foi validado por métodos de Propensity Score Matching, que fazem menos suposições sobre os dados do que uma Regressão Linear padrão.


